<a href="https://colab.research.google.com/github/MdSourav76046/AI_Lab-4.1/blob/main/My_Thesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **This code will connect my model with the google vision OCR**

In [ ]:
import os

# Replace 'source.json' with your actual JSON file name
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/drive/MyDrive/google_ocr/source.json"


In [ ]:
!pip install google-cloud-vision
from google.cloud import vision

# Initialize the Vision API client
try:
    client = vision.ImageAnnotatorClient()
    print("Google Vision API authenticated successfully!")
except Exception as e:
    print(f"Authentication failed: {e}")


Google Vision API authenticated successfully!


## **This code will extract the text from the image using google vision OCR**

In [ ]:
from google.colab import files
from PIL import Image
import io
import os
# Import the necessary library
from google.cloud import vision


# Set up authentication using the uploaded JSON key file
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/drive/MyDrive/google_ocr/source.json"

# Initialize the Vision API client
client = vision.ImageAnnotatorClient()

# Function to extract text from an image
def extract_text_from_image(image_path):
    with io.open(image_path, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    response = client.text_detection(image=image)
    texts = response.text_annotations

    if not texts:
        return "No text detected in the image."

    # Extract bounding boxes and text
    text_data = []
    for text in texts[1:]:  # Skip the first entry (it includes all the text)
        bounding_box = text.bounding_poly.vertices
        top_left_y = bounding_box[0].y
        top_left_x = bounding_box[0].x
        text_data.append((top_left_y, top_left_x, text.description))

    # Sort the text data by y-coordinate, then x-coordinate
    text_data.sort(key=lambda x: (x[0], x[1]))

    # Combine the sorted text
    sorted_text = " ".join([item[2] for item in text_data])
    return sorted_text

# Upload the image to be processed
uploaded_image = files.upload()

# Get the uploaded image file name
image_name = list(uploaded_image.keys())[0]

# Extract text from the uploaded image
extracted_text = extract_text_from_image(image_name)

# Print the extracted text
print("Extracted Text:")
print(extracted_text)


Saving IMG_1739.jpeg to IMG_1739 (2).jpeg
Extracted Text:
দুই হাজার ঢাকা দিতছি " " 1 জিতছি টাকা হাজার দুই জিতছি টাকা হাজার দুই জিতছি টাকা হাজার দুই


## **This code will then try to detect the gramartical error in the code and it will correct the grammer.**

In [ ]:
import requests

def correct_text_with_languagetool(text):

    # LanguageTool API endpoint
    url = "https://api.languagetoolplus.com/v2/check"

    # API payload
    payload = {
        "text": text,
        "language": "en-US"
    }

    # Make the API request
    response = requests.post(url, data=payload)
    result = response.json()

    # Apply the corrections to the original text
    corrected_text = text
    if "matches" in result:
        for match in reversed(result["matches"]):  # Process matches in reverse order
            start = match["offset"]
            end = start + match["length"]
            replacement = match["replacements"][0]["value"] if match["replacements"] else ""
            corrected_text = corrected_text[:start] + replacement + corrected_text[end:]

    return corrected_text

# Example usage
input_text = "I going t homeeee"
corrected_text = correct_text_with_languagetool(input_text)

print("Original Text:")
print(input_text)
print("\nCorrected Text:")
print(corrected_text)


Original Text:
I going t homeeee

Corrected Text:
I'm going t homered


## Improve the accuracy for handwritting text

In [ ]:
!pip install google-cloud-documentai spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 65.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import cv2
import io
import os
from google.cloud import documentai_v1 as documentai
import matplotlib.pyplot as plt
import spacy

# ✅ Step 1: Extract Text and Bounding Boxes with Document AI
def extract_text_with_boxes(project_id, location, processor_id, file_path, json_key_path):

    # Create client using the JSON key file
    client = documentai.DocumentProcessorServiceClient.from_service_account_json(json_key_path)

    # Construct the resource name of the processor
    name = f"projects/{project_id}/locations/{location}/processors/{processor_id}"

    # Read the file
    with io.open(file_path, "rb") as image_file:
        content = image_file.read()

    # Create a raw document object
    raw_document = documentai.RawDocument(content=content, mime_type="image/jpeg")

    # Configure the request
    request = documentai.ProcessRequest(name=name, raw_document=raw_document)

    # Process the document
    result = client.process_document(request=request)

    # Extract text and bounding box information
    document = result.document
    text_content = document.text
    elements = []

    for page in document.pages:
        for paragraph in page.paragraphs:
            if paragraph.layout.bounding_poly:
                bbox = paragraph.layout.bounding_poly.vertices
                vertices = [(vertex.x, vertex.y) for vertex in bbox]
                text = paragraph.layout.text_anchor.text_segments
                paragraph_text = "".join(
                    [text_content[seg.start_index:seg.end_index] for seg in text]
                )
                elements.append({"text": paragraph_text, "bbox": vertices})

    return elements


# ✅ Step 2: Reorder Text Using Bounding Boxes
def reorder_text(elements):
    # Sort elements by Y-coordinate (top to bottom) and X-coordinate (left to right)
    elements.sort(key=lambda e: (min([v[1] for v in e["bbox"]]), min([v[0] for v in e["bbox"]])))

    # Combine text in reading order
    reordered_text = "\n".join([e["text"] for e in elements])
    return reordered_text


# ✅ Step 3: Use NLP for Refinement
def refine_text_with_nlp(text):

    nlp = spacy.load("en_core_web_sm")  # Load English NLP model
    doc = nlp(text)

    # Combine sentences into refined paragraphs
    refined_text = " ".join([sent.text.strip() for sent in doc.sents])
    return refined_text


# ✅ Step 4: Complete Pipeline
def process_image_with_context(project_id, location, processor_id, file_path, json_key_path):

    # Extract text and bounding boxes
    elements = extract_text_with_boxes(project_id, location, processor_id, file_path, json_key_path)

    # Reorder text based on bounding boxes
    reordered_text = reorder_text(elements)

    # Refine text using NLP
    final_text = refine_text_with_nlp(reordered_text)
    return final_text


# ✅ Step 5: Configure API Credentials and Run Pipeline
PROJECT_ID = "corded-skill-446022-r2"
LOCATION = "us"  # Use 'us' or 'eu' based on your processor's location
PROCESSOR_ID = "4d451d213280960"  # Replace with your processor ID
IMAGE_PATH = "/content/drive/MyDrive/Pic_for_Research/myHandWriting2.JPG"  # Replace with your image path
JSON_KEY_PATH = "/content/drive/MyDrive/google_document_ai/documentAi.json"  # Replace with your JSON key path

# Process the image and extract refined text
final_text = process_image_with_context(PROJECT_ID, LOCATION, PROCESSOR_ID, IMAGE_PATH, JSON_KEY_PATH)

# Print the final refined text
print("\nFinal Extracted and Refined Text:")
print(final_text)


/usr/local/lib/python3.11/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)



Final Extracted and Refined Text:
I want to be

they

man. untill they

good

لی

nullify

my strength. me

and


## Code with NLP + OCR + Bounding box

In [ ]:
import io
import os
import spacy
from google.cloud import documentai_v1 as documentai

# ✅ Step 1: Extract Text with Bounding Box Positions
def extract_text_with_bounding_boxes(project_id, location, processor_id, file_path, json_key_path):
    """
    Extracts text along with bounding box positions from an image using Google Document AI.

    Args:
        project_id (str): Google Cloud project ID.
        location (str): Location of the processor ('us' or 'eu').
        processor_id (str): ID of the Document AI processor.
        file_path (str): Path to the image.
        json_key_path (str): Path to the service account JSON key file.

    Returns:
        list: Extracted words with bounding box positions.
    """
    client = documentai.DocumentProcessorServiceClient.from_service_account_json(json_key_path)
    name = f"projects/{project_id}/locations/{location}/processors/{processor_id}"

    with io.open(file_path, "rb") as image_file:
        content = image_file.read()

    raw_document = documentai.RawDocument(content=content, mime_type="image/jpeg")
    request = documentai.ProcessRequest(name=name, raw_document=raw_document)
    result = client.process_document(request=request)

    document = result.document
    extracted_data = []

    for page in document.pages:
        for paragraph in page.paragraphs:
            if paragraph.layout.bounding_poly:
                bbox = paragraph.layout.bounding_poly.vertices
                vertices = [(vertex.x, vertex.y) for vertex in bbox]
                text = paragraph.layout.text_anchor.text_segments
                paragraph_text = "".join(
                    [document.text[seg.start_index:seg.end_index] for seg in text]
                )
                extracted_data.append({"text": paragraph_text, "bbox": vertices})

    return extracted_data


# ✅ Step 2: Reorder Text Based on Bounding Box Positions
def reorder_text(extracted_data):
    """
    Reorders words based on bounding box positions for correct sentence flow.

    Args:
        extracted_data (list): List of extracted words with bounding box positions.

    Returns:
        str: Properly ordered text.
    """
    # Sort words by Y-coordinate (to ensure correct line ordering), then by X (left to right)
    extracted_data.sort(key=lambda e: (min(v[1] for v in e["bbox"]), min(v[0] for v in e["bbox"])))

    # Combine words into sentences
    ordered_text = " ".join([e["text"] for e in extracted_data])
    return ordered_text


# ✅ Step 3: Fix Grammar & Sentence Structure Using NLP
def refine_text_with_nlp(text):
    """
    Refines the extracted text using NLP to improve sentence flow.

    Args:
        text (str): Raw extracted text.

    Returns:
        str: Refined text with corrected grammar.
    """
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)

    refined_text = " ".join([sent.text.strip() for sent in doc.sents])
    return refined_text


# ✅ Step 4: Full Pipeline - Correct Text Order from OCR
def process_image_for_correct_text_order(project_id, location, processor_id, file_path, json_key_path):
    """
    Extracts text from an image, reorders it using bounding box positions, and refines grammar using NLP.

    Args:
        project_id (str): Google Cloud project ID.
        location (str): Location of the processor ('us' or 'eu').
        processor_id (str): ID of the Document AI processor.
        file_path (str): Path to the image.
        json_key_path (str): Path to the service account JSON key file.

    Returns:
        str: Final corrected text.
    """
    print("🔍 Extracting text with bounding boxes...")
    extracted_data = extract_text_with_bounding_boxes(project_id, location, processor_id, file_path, json_key_path)

    print("🔄 Reordering text based on bounding boxes...")
    ordered_text = reorder_text(extracted_data)

    print("📝 Refining text using NLP for better readability...")
    final_text = refine_text_with_nlp(ordered_text)

    return final_text


# ✅ Step 5: Configure API Credentials and Run Pipeline
PROJECT_ID = "corded-skill-446022-r2"
LOCATION = "us"  # Use 'us' or 'eu' based on your processor's location
PROCESSOR_ID = "4d451d213280960"  # Replace with your processor ID
IMAGE_PATH = "/content/drive/MyDrive/Pic_for_Research/myHandWriting2.JPG"  # Replace with your image path
JSON_KEY_PATH = "/content/drive/MyDrive/google_document_ai/documentAi.json"  # Replace with your JSON key path

# ✅ Run the OCR Correction Pipeline
corrected_text = process_image_for_correct_text_order(PROJECT_ID, LOCATION, PROCESSOR_ID, IMAGE_PATH, JSON_KEY_PATH)

# ✅ Step 6: Print the Final Corrected Text
print("\n✨ Final Corrected Text:")
print(corrected_text)


🔍 Extracting text with bounding boxes...
🔄 Reordering text based on bounding boxes...
📝 Refining text using NLP for better readability...

✨ Final Corrected Text:
I want to be
 man. untill they
 they
 good
 لی
 nullify
 my strength. me
 and


## Working with NLP

In [ ]:
import random
import pandas as pd
from tqdm import tqdm

# ✅ Base correct sentences (expand this list for more variety)
correct_sentences = [
    "I want to be a good person.",
    "The weather is nice today.",
    "He is studying machine learning.",
    "She loves reading books in the evening.",
    "Artificial intelligence is transforming the world.",
    "Python is my favorite programming language.",
    "We need to prepare for the final exam.",
    "The cat is sitting on the windowsill.",
    "Tomorrow we will visit the museum.",
    "Learning new skills is important for personal growth.",
    "He enjoys watching football on weekends.",
    "The sun is shining brightly in the sky.",
    "They are planning a trip to the mountains.",
    "I have an important meeting at 10 AM.",
    "Exercising regularly keeps you healthy.",
    "She bought a beautiful dress for the party.",
    "We should always be kind to others.",
    "The coffee shop is located near the library.",
    "He is writing an article about climate change.",
    "Traveling helps people understand different cultures.",
    "Mathematics is essential for engineering and science.",
    "She practices yoga every morning to stay fit.",
    "They adopted a puppy from the animal shelter.",
    "I need to finish my assignment before the deadline.",
    "History teaches us valuable lessons about the past.",
    "Cooking at home is healthier than eating out.",
    "Technology is advancing at a rapid pace.",
    "He is trying to learn a new language.",
    "The artist is painting a beautiful landscape.",
    "Reading helps improve vocabulary and comprehension.",
    "I love to read books on weekends.",
    "The weather is beautiful today.",
    "He is learning how to code in Python.",
    "We are planning a trip to the mountains.",
    "She enjoys baking cakes in her free time.",
    "The sun is shining brightly in the sky.",
    "I want to be a successful person.",
    "They are watching a movie together.",
    "Learning new skills is important for personal growth.",
    "He enjoys playing football with his friends.",
    "She writes poems about nature and life.",
    "The cat is sitting on the windowsill.",
    "We should always be kind to others.",
    "He is preparing for his final exams.",
    "The train arrives at the station in the evening.",
    "She practices yoga every morning to stay fit.",
    "They adopted a puppy from the animal shelter.",
    "History teaches us valuable lessons about the past.",
    "Cooking at home is healthier than eating out.",
    "Technology is advancing at a rapid pace.",
    "The artist is painting a beautiful landscape.",
    "The students are studying for their math test.",
    "The coffee shop is located near the library.",
    "Traveling helps people understand different cultures.",
    "Exercising regularly keeps you healthy and active.",
]

# ✅ Generate 500 shuffled sentence pairs
dataset = []
for _ in tqdm(range(5000)):  # Create 500 unique data points
    sentence = random.choice(correct_sentences)  # Pick a random sentence
    words = sentence.split()
    shuffled_words = words.copy()
    random.shuffle(shuffled_words)  # Shuffle words randomly
    shuffled_sentence = " ".join(shuffled_words)

    dataset.append({"input": f"reorder: {shuffled_sentence}", "output": sentence})

# ✅ Convert dataset to DataFrame
df = pd.DataFrame(dataset)

# ✅ Save dataset as CSV for future use
df.to_csv("sentence_reordering_dataset.csv", index=False)

print("✅ 500-Sentence Dataset Created & Saved!")


100%|██████████| 5000/5000 [00:00<00:00, 90074.18it/s]


✅ 500-Sentence Dataset Created & Saved!


In [ ]:
# ✅ Load and print 5 random examples
df = pd.read_csv("sentence_reordering_dataset.csv")
print(df.sample(5))  # Show 5 random sentences


                                                    input  \
179061     reorder: is cat sitting the windowsill. The on   
289002   reorder: The brightly the sun in shining is sky.   
82403   reorder: at is healthier home out. eating than...   
191726  reorder: from puppy animal shelter. They a the...   
118234  reorder: advancing Technology at is a rapid pace.   

                                               output  
179061          The cat is sitting on the windowsill.  
289002        The sun is shining brightly in the sky.  
82403   Cooking at home is healthier than eating out.  
191726  They adopted a puppy from the animal shelter.  
118234       Technology is advancing at a rapid pace.  


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch
import pandas as pd
from tqdm import tqdm

# ✅ Load dataset
df = pd.read_csv("sentence_reordering_dataset.csv")
train_data = df.to_dict("records")

# ✅ Load pre-trained T5 model & tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# ✅ Tokenize input and output
input_texts = [tokenizer(item["input"], return_tensors="pt", padding=True, truncation=True) for item in train_data]
output_texts = [tokenizer(item["output"], return_tensors="pt", padding=True, truncation=True) for item in train_data]

# ✅ Set up optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

# ✅ Train for 10 epochs on 500 sentences
print("🔄 Training Model on Sentence Reordering...")
for epoch in range(25):
    total_loss = 0
    for inp, out in tqdm(zip(input_texts, output_texts), total=len(input_texts)):
        outputs = model(**inp, labels=out["input_ids"])
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

# ✅ Save fine-tuned model
model.save_pretrained("t5_sentence_reorder")
tokenizer.save_pretrained("t5_sentence_reorder")
print("✅ Model Training Complete & Saved!")


🔄 Training Model on Sentence Reordering...


100%|██████████| 5000/5000 [1:22:36<00:00,  1.01it/s]


Epoch 1, Loss: 236.7699


100%|██████████| 5000/5000 [1:24:13<00:00,  1.01s/it]


Epoch 2, Loss: 0.4762


 84%|████████▍ | 4194/5000 [1:12:54<13:16,  1.01it/s]

In [ ]:
# ✅ Load Fine-Tuned Model for Inference
model = T5ForConditionalGeneration.from_pretrained("t5_sentence_reorder")
tokenizer = T5Tokenizer.from_pretrained("t5_sentence_reorder")

def correct_sentence_t5(input_text):
    """
    Uses fine-tuned T5 model to correct OCR sentence order.
    """
    input_text = "reorder: " + input_text
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    output_ids = model.generate(input_ids, max_length=50)
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

# ✅ Test the model with NEW, unseen sentences
test_sentences = [
    "i be want man a good",
    "mother is name my Shilpi",
    "much love i her love so",
    "how he to it do knows"
]

print("\n🔹 Example Predictions:")
for sentence in test_sentences:
    print(f"📝 Input: {sentence}")
    print(f"✅ Output: {correct_sentence_t5(sentence)}\n")


NameError: name 'T5ForConditionalGeneration' is not defined

## Working with LSTM Based pointer network

In [ ]:
!pip install torch numpy pandas tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 25.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
import pandas as pd

# ✅ Define Correctly Ordered Sentences
correct_sentences = [
    "I want to be a good man",
    "My mother's name is Shilpi",
    "I love her so much",
    "He knows how to do it",
    "The sun is shining brightly today",
    "She enjoys baking cakes in her free time",
    "Technology is advancing at a rapid pace",
]

# ✅ Generate Training Data (Shuffled Sentences)
def generate_training_data(sentences, num_samples=500):
    training_data = []
    for _ in range(num_samples):
        sentence = random.choice(sentences)  # Pick a correct sentence
        words = sentence.split()
        shuffled_words = words.copy()
        random.shuffle(shuffled_words)  # Shuffle words randomly
        training_data.append({"input": shuffled_words, "output": words})
    return training_data

# ✅ Create dataset
dataset = generate_training_data(correct_sentences, num_samples=500)

# ✅ Convert to DataFrame and Save
df = pd.DataFrame(dataset)
df.to_csv("sentence_reordering_dataset.csv", index=False)
print("✅ Dataset Created and Saved!")


✅ Dataset Created and Saved!


In [ ]:
# ✅ Load Dataset
df = pd.read_csv("sentence_reordering_dataset.csv")

# ✅ Build Vocabulary
unique_words = set(word for sentence in df["input"] for word in sentence.split())
word2idx = {word: idx + 1 for idx, word in enumerate(unique_words)}  # +1 for padding
idx2word = {idx: word for word, idx in word2idx.items()}

# ✅ Add <UNK> Token for Unseen Words
word2idx["<UNK>"] = len(word2idx) + 1
idx2word[len(word2idx)] = "<UNK>"

print("✅ Vocabulary Size:", len(word2idx))


✅ Vocabulary Size: 115


In [ ]:
# ✅ Function to encode sentences safely
def encode_sentence(sentence, word2idx):
    return [word2idx.get(word, word2idx["<UNK>"]) for word in sentence.split()]

# ✅ Encode input and output sentences
df["input_encoded"] = df["input"].apply(lambda x: encode_sentence(x, word2idx))
df["output_encoded"] = df["output"].apply(lambda x: encode_sentence(x, word2idx))

# ✅ Get max sequence length
max_length = max(df["input_encoded"].apply(len))

# ✅ Function to pad sequences
def pad_sequence(seq, max_length):
    return seq + [0] * (max_length - len(seq))  # Pad with 0s

# ✅ Apply padding
df["input_encoded"] = df["input_encoded"].apply(lambda x: pad_sequence(x, max_length))
df["output_encoded"] = df["output_encoded"].apply(lambda x: pad_sequence(x, max_length))

# ✅ Convert to PyTorch tensors
X_train = torch.tensor(df["input_encoded"].tolist(), dtype=torch.long)
y_train = torch.tensor(df["output_encoded"].tolist(), dtype=torch.long)

print("✅ Data Processed Successfully! Training Samples:", len(X_train))


✅ Data Processed Successfully! Training Samples: 500


In [ ]:
class PointerLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(PointerLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.attention = nn.Linear(hidden_dim * 2, max_length)  # Attention layer
        self.output_layer = nn.Linear(hidden_dim * 2, vocab_size)

    def forward(self, x):
        embedded = self.embedding(x)  # Convert words to embeddings
        lstm_out, _ = self.lstm(embedded)  # Pass through LSTM

        # Compute attention scores
        attention_scores = torch.softmax(self.attention(lstm_out), dim=1)

        # Compute output using weighted sum
        output = torch.matmul(attention_scores.transpose(1, 2), lstm_out)
        output = self.output_layer(output)

        return output, attention_scores


In [ ]:
# ✅ Model Parameters
embedding_dim = 64
hidden_dim = 128
vocab_size = len(word2idx) + 1  # +1 for padding
model = PointerLSTM(vocab_size, embedding_dim, hidden_dim)

# ✅ Training Parameters
epochs = 25
learning_rate = 0.005

# ✅ Define Loss and Optimizer
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# ✅ Training Loop
for epoch in range(epochs):
    total_loss = 0
    for i in range(len(X_train)):
        optimizer.zero_grad()

        # Forward pass
        input_seq = X_train[i].unsqueeze(0)
        target_seq = y_train[i].unsqueeze(0)
        output, _ = model(input_seq)

        loss = criterion(output.view(-1, vocab_size), target_seq.view(-1))
        total_loss += loss.item()

        # Backpropagation
        loss.backward()
        optimizer.step()

    print(f"✅ Epoch {epoch+1}, Loss: {total_loss / len(X_train):.4f}")


✅ Epoch 1, Loss: 0.8349
✅ Epoch 2, Loss: 0.1017
✅ Epoch 3, Loss: 0.0427
✅ Epoch 4, Loss: 0.0491
✅ Epoch 5, Loss: 0.0628
✅ Epoch 6, Loss: 0.0826
✅ Epoch 7, Loss: 0.0612
✅ Epoch 8, Loss: 0.0325
✅ Epoch 9, Loss: 0.0324
✅ Epoch 10, Loss: 0.0335
✅ Epoch 11, Loss: 0.0275
✅ Epoch 12, Loss: 0.0308
✅ Epoch 13, Loss: 0.1062
✅ Epoch 14, Loss: 0.0726
✅ Epoch 15, Loss: 0.0500
✅ Epoch 16, Loss: 0.0303
✅ Epoch 17, Loss: 0.0293
✅ Epoch 18, Loss: 0.0107
✅ Epoch 19, Loss: 0.0163
✅ Epoch 20, Loss: 0.0506
✅ Epoch 21, Loss: 0.0441
✅ Epoch 22, Loss: 0.0254
✅ Epoch 23, Loss: 0.0177
✅ Epoch 24, Loss: 0.0206
✅ Epoch 25, Loss: 0.0053


In [1]:
!git remote add origin https://github.com/MdSourav76046/My_Thesis.git
!git branch -M main
!git push -u origin main

fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
